In [1]:
import numpy as np
import pandas as pd
import pandasql as psql
import matplotlib.pyplot as plt
import seaborn as sns
from time import time

from sklearn.metrics import mean_squared_error, mean_absolute_error

import numpy as np
from scipy.spatial.distance import cdist

import gc

seed=15

# Lectura de los datos y embeddings

In [2]:
%%time
df_train_orig = pd.read_parquet('../data/df_filt_train_100_100.zip')
df_train_orig.shape

Wall time: 974 ms


(8556951, 3)

In [3]:
df_train_orig.head(10)

,movie_id,user_id,rating
10930635,2128,2000429,4
46725692,8387,382224,3
27446119,5075,1617455,4
80443653,14491,143113,3
85964770,15246,768876,5
80408073,14484,2261348,4
24386603,4577,2451473,3
88071659,15657,2364018,3
91429355,16265,2110908,5
11491757,2186,662478,3


In [4]:
%%time
df_val_orig = pd.read_parquet('../data/df_filt_val_100_100.zip')
df_val_orig.shape

Wall time: 413 ms


(6417714, 3)

In [5]:
df_val_orig.head(10)

,movie_id,user_id,rating
3884480,752,1551333,5
83843802,14961,2009898,4
24357576,4577,168479,5
49181371,8846,1908261,4
53710780,9757,659928,3
85028019,15116,2339843,4
10697378,2108,631598,5
49339509,8880,1712681,4
85932015,15234,756211,2
13763540,2617,2030407,3


In [6]:
df_trainval_orig = pd.concat([df_train_orig,df_val_orig])

In [7]:
df_trainval_orig.shape

(14974665, 3)

In [8]:
%%time
df_test_orig = pd.read_parquet('../data/df_filt_test_100_100.zip')
df_test_orig.shape

Wall time: 390 ms


(6417714, 3)

In [9]:
df_test_orig.head(10)

,movie_id,user_id,rating
32205464,5862,98185,5
92053487,16349,540005,3
49376257,8883,1967541,3
40067268,7067,439729,2
15903334,3098,1484284,3
65188281,11907,2304849,2
85978538,15257,1990901,4
91305866,16242,1562812,4
75978374,13748,1848739,3
94011219,16698,2630337,5


Leemos las traducciones de los IDs para los embeddings

In [10]:
translate_movies = pd.read_csv('translate_movies_tts_100_100.csv')
translate_movies.head()

,movie_id,movie_id_orig
0,3588,2128
1,6336,8387
2,4885,5075
3,1987,14491
4,2323,15246


In [11]:
translate_users = pd.read_csv('translate_users_tts_100_100.csv')
translate_users.head()

,user_id,user_id_orig
0,31206,2000429
1,55088,382224
2,19393,1617455
3,13510,143113
4,67063,768876


In [12]:
'''query_df_trainval_translate = """
SELECT b.movie_id
    , c.user_id
    , a.rating
    , a.movie_id as movie_id_orig
    , a.user_id as user_id_orig

from df_trainval_orig as a

left join translate_movies as b
    on a.movie_id = b.movie_id_orig

left join translate_users as c
    on a.user_id = c.user_id_orig

"""
'''

query_df_trainval_translate = """
SELECT b.movie_id
    , c.user_id
    , a.rating

from df_trainval_orig as a

left join translate_movies as b
    on a.movie_id = b.movie_id_orig

left join translate_users as c
    on a.user_id = c.user_id_orig

"""

In [13]:
%%time
df_trainval = psql.sqldf(query_df_trainval_translate, locals())
df_trainval.head()

Wall time: 4min 42s


,movie_id,user_id,rating
0,3588,31206,4
1,6336,55088,3
2,4885,19393,4
3,1987,13510,3
4,2323,67063,5


In [14]:
df_trainval.shape

(14974665, 3)

In [15]:
query_df_test_translate = """
SELECT b.movie_id
    , c.user_id
    , a.rating

from df_test_orig as a

left join translate_movies as b
    on a.movie_id = b.movie_id_orig

left join translate_users as c
    on a.user_id = c.user_id_orig

"""

In [16]:
%%time
df_test = psql.sqldf(query_df_test_translate, locals())
df_test.head()

Wall time: 1min 24s


,movie_id,user_id,rating
0,5206,73850,5
1,2816,60041,3
2,6568,30180,3
3,5744,56978,2
4,4013,15170,3


In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6417714 entries, 0 to 6417713
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   movie_id  int64
 1   user_id   int64
 2   rating    int64
dtypes: int64(3)
memory usage: 146.9 MB


In [18]:
df_test.shape

(6417714, 3)

In [19]:
# cargamos los embeddings
emb_dim = 10
lr_str = 20
niter_str = "10k"

modelo = "embeddings/"
tamanyo = f"_dim{emb_dim}_{lr_str}_{niter_str}"

print(f'./{modelo}user_embeddings{tamanyo}.npy')
print(f'./{modelo}movie_embeddings{tamanyo}.npy')

user_embeddings = np.load(f'./{modelo}user_embeddings{tamanyo}.npy')
movie_embeddings = np.load(f'./{modelo}movie_embeddings{tamanyo}.npy')

./embeddings/user_embeddings_dim10_20_10k.npy
./embeddings/movie_embeddings_dim10_20_10k.npy


# Funciones de predicción

In [20]:
def predict_dot_product(user_id, movie_id, user_embeddings=user_embeddings, movie_embeddings=movie_embeddings):
    return np.dot(user_embeddings[user_id],movie_embeddings[movie_id])

In [21]:
def create_index(df_input, var):
    """
    Crea un diccionario que mapea cada user_id a los índices de las películas que ha visto.

    Args:
        df_input (pd.DataFrame): DataFrame con columnas ['user_id', 'movie_id', 'rating'].

    Returns:
        dict: Diccionario con user_id como clave y lista de índices como valor.
    """
    indices = {}
    for var_id, group in df_input.groupby(var):
        indices[var_id] = group.index.tolist()
    return indices


In [22]:
%%time
user_index = create_index(df_trainval,"user_id")
movie_index = create_index(df_trainval,"movie_id")

Wall time: 8.09 s


In [27]:
def predict_with_k_nearest_movies(target_movie_id, user_id, k=10, movie_embeddings=movie_embeddings, df_input=df_trainval, user_index=user_index):
    """
    Predice la calificación de una película objetivo para un usuario dado basándose en las k-nearest películas calificadas por el usuario.

    Args:
        target_movie_id (int): ID de la película objetivo para la que queremos predecir la calificación.
        user_id (int): ID del usuario para quien estamos prediciendo la calificación.
        k (int): Valores para k (número de vecinos más cercanos) a usar para la predicción.
        movie_embeddings (np.array): Array de embeddings de películas donde el índice corresponde a movie_id.
        df_input (pd.DataFrame): DataFrame con columnas ['user_id', 'movie_id', 'rating'] con calificaciones de usuarios.
        user_index (dict): Diccionario con user_id como clave y lista de índices de las películas que ha visto como valor.
        
    Returns:
        pred: Calificacion predicha.
    """

    # Filtrar las películas calificadas por el usuario (usando los índices en lugar de aplicar condiciones)
    df_filt = df_input.iloc[user_index[user_id]][['movie_id', 'rating']]
    movies_ids = df_filt['movie_id'].values

    # Calcular las distancias entre la película objetivo y las películas calificadas por el usuario
    distances = cdist(movie_embeddings[movies_ids], movie_embeddings[target_movie_id].reshape(1, -1)).flatten()
    distances_argsort = np.argsort(distances)

    # Precalcular las inversas de las distancias para los vecinos más cercanos
    inverse_distances = 1 / distances[distances_argsort]

    nearest_inverse_distances = inverse_distances[:k]
    nearest_ratings = df_filt.iloc[distances_argsort[:k], 1].values

    # Predicción ponderada por la inversa de las distancias
    pred = np.dot(nearest_inverse_distances, nearest_ratings) / nearest_inverse_distances.sum()

    return pred


In [28]:
def predict_with_k_nearest_users(target_user_id, movie_id, k=10, user_embeddings=user_embeddings, df_input=df_trainval, movie_index=movie_index):
    """
    Predice la calificación de una película objetivo para un usuario dado basándose en los k usuarios más cercanos
    que han calificado la película.

    Args:
        target_user_id (int): ID del usuario objetivo para el cual queremos predecir la calificación.
        movie_id (int): ID de la película para la cual estamos prediciendo la calificación.
        ks (list of int): Lista de valores para k (número de vecinos más cercanos) a usar para la predicción.
        user_embeddings (np.array): Array de embeddings de usuarios donde el índice corresponde a user_id.
        df_input (pd.DataFrame): DataFrame que contiene las columnas ['user_id', 'movie_id', 'rating'] con calificaciones de usuarios.
        movie_index (dict): Diccionario con movie_id como clave y lista de índices de los usuarios que han calificado esa película como valor.

    Returns:
        pred: Calificacion predicha.
    """

    # Filtrar los usuarios que han calificado la película (usando los índices en lugar de aplicar condiciones)
    df_filt = df_input.iloc[movie_index[movie_id]][['user_id', 'rating']]
    users_ids = df_filt['user_id'].values

    # Calcular las distancias entre el usuario objetivo y los usuarios que han calificado la película
    distances = cdist(user_embeddings[users_ids], user_embeddings[target_user_id].reshape(1, -1)).flatten()
    distances_argsort = np.argsort(distances)

    # Precalcular las inversas de las distancias para los vecinos más cercanos
    inverse_distances = 1 / distances[distances_argsort]

    nearest_inverse_distances = inverse_distances[:k]
    nearest_ratings = df_filt.iloc[distances_argsort[:k], 1].values

    # Predicción ponderada por la inversa de las distancias
    pred = np.dot(nearest_inverse_distances, nearest_ratings) / nearest_inverse_distances.sum()

    return pred


# Puntuación del test

In [29]:
%%time
# CUIDADO QUE ESTE TARDA
k_movie = 6
k_users = 7

gc.collect()

t0 = time()
df_test['prediction_movies'] = df_test.apply(lambda row: predict_with_k_nearest_movies(int(row['movie_id']), int(row['user_id']), k=k_movie), axis=1)
t1 = time()
print(f"predict_with_k_nearest_movies: {(t1-t0)/60} mins\n")

gc.collect()

t0 = time()
df_test['prediction_users']  = df_test.apply(lambda row: predict_with_k_nearest_users(int(row['user_id']), int(row['movie_id']), k=k_users), axis=1)
t1 = time()
print(f"predict_with_k_nearest_users:  {(t1-t0)/60} mins\n")

gc.collect()

t0 = time()
df_test['prediction_dot'] = df_test.apply(lambda row: predict_dot_product(int(row['user_id']), int(row['movie_id'])), axis=1)
t1 = time()
print(f"df_predict_dot_product: {(t1-t0)/60} mins")

gc.collect()

predict_with_k_nearest_movies: 137.3395547469457 mins

predict_with_k_nearest_users:  755.3119462132454 mins

df_predict_dot_product: 2.622064391771952 mins
Wall time: 14h 55min 18s


0

In [31]:
df_test.to_parquet('../data/df_filt_test_100_100_punt.zip',compression='gzip')

In [35]:
%%time
# Calculo de las metricas
t0=time()

metricas_movies1 = mean_absolute_error(y_true=df_test['rating'], y_pred=df_test[f'prediction_movies'])
metricas_users1 = mean_absolute_error(y_true=df_test['rating'], y_pred=df_test[f'prediction_users'])
metricas_dot1 = mean_absolute_error(y_true=df_test['rating'], y_pred=df_test['prediction_dot'])

t1=time()
print(f"PUNTO 2: TIEMPO {t1-t0}")
print(f"MAE movie: {metricas_movies1}")
print(f"MAE users: {metricas_users1}")
print(f"MAE dot:   {metricas_dot1}")
print()
t0=time()

metricas_movies2 = mean_squared_error(y_true=df_test['rating'], y_pred=df_test[f'prediction_movies'])**0.5
metricas_users2 = mean_squared_error(y_true=df_test['rating'], y_pred=df_test[f'prediction_users'])**0.5
metricas_dot2 = mean_squared_error(y_true=df_test['rating'], y_pred=df_test['prediction_dot'])**0.5

t1=time()
print(f"PUNTO 3: TIEMPO {t1-t0}")
print(f"RMSE movie: {metricas_movies2}")
print(f"RMSE users: {metricas_users2}")
print(f"RMSE dot:   {metricas_dot2}")
print()
t0=time()

metricas_movies3 = (df_test[f'prediction_movies'] - df_test['rating']).mean()
metricas_users3 = (df_test[f'prediction_users'] - df_test['rating']).mean()
metricas_dot3 = (df_test[f'prediction_dot'] - df_test['rating']).mean()

t1=time()
print(f"PUNTO 4: TIEMPO {t1-t0}")
print(f"ME movie: {metricas_movies3}")
print(f"ME users: {metricas_users3}")
print(f"ME dot:   {metricas_dot3}")
print()

PUNTO 2: TIEMPO 0.18988418579101562
MAE movie: 0.741120166619776
MAE users: 0.8369547226571441
MAE dot:   0.7252175457289387

PUNTO 3: TIEMPO 0.16689705848693848
RMSE movie: 0.9573234999623015
RMSE users: 1.0492692721790036
RMSE dot:   0.926698879171664

PUNTO 4: TIEMPO 0.07994842529296875
ME movie: 0.009340945393179754
ME users: -0.020291249176279447
ME dot:   -0.0031215966588185786

Wall time: 437 ms
